In [ ]:
cd /home/schlecker/repos/planeteScripts

In [292]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.optimize import curve_fit

import output
import plots
import stats
import utils

%load_ext autoreload
%autoreload 2

# default output folder
# outputs = '/home/schlecker/phd/planete/outputs/pop06_MstarGrid/'
outputs = '/media/martin/Daten/phd/planete/outputs/pop06_MstarGrid/'

pd.set_option('display.max_columns', 500)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Read ref_reds into a list of populations

In [ ]:
filenames = ['ref_red5e9_0.3Msol.dat', 'ref_red5e9_0.5Msol.dat', 'ref_red5e9_0.7Msol.dat', 'ref_red5e9_1.0Msol.dat'] 
populations = [output.Population(outputs + f, name=f[11:18]) for f in filenames]

In [ ]:
pops = output.Population([outputs + f for f in filenames])

concatenate population data into a single multiindex-dataframe

In [ ]:
jointdf = pd.concat([pop.data for pop in populations], axis=0, keys=[pop.name for pop in populations])

not yet sure what to do with this... lets get some stats

In [ ]:
stats = {p.name : p.get_typeStats() for p in populations}

In [ ]:
stats['0.5Msol']

In [ ]:
for planetType in stats['0.5Msol']:
    print(planetType)

how about a summary plot?

In [ ]:
def plot_typeStats(populations):
    """
    
    """
    if isinstance(populations, list):
        stats = {p.name : p.get_typeStats() for p in populations}
        fig, ax = plt.subplots()
        stellarMasses = [0.3, 0.5, 0.7, 1.0]
        for M, Mstar in enumerate(stats.keys()):
            for planetType in stats[Mstar]:
                plt.errorbar(stellarMasses[M], stats[Mstar][planetType]['meanMetallicity'], stats[Mstar][planetType]['stdMetallicity'],
                             capsize=40., fmt='o', label=planetType)
        plt.legend()

In [ ]:
plot_typeStats(populations)

wow, this was rough. let's try something else

In [ ]:
for Mstar, MPop in jointdf.groupby(level=0):

In [ ]:
sns.catplot(x='Mstar', y='Metallicity', kind='bar', data=jointdf)

## disk lifetime analysis

In [293]:
tDiskFiles = [outputs + s for s in ['ref_redtdisk_0.3Msol.dat','ref_redtdisk_0.5Msol.dat', 'ref_redtdisk_0.7Msol.dat', 'ref_redtdisk_1.0Msol.dat']]

In [294]:
Mgrid = output.Population()
Mgrid.read_tDiskData(tDiskFiles)

compute disk fractions and store them separately for each Mstar

In [373]:
diskFractions = {Mstar : utils.get_diskFractions(tDiskData, Ntimes=50) for Mstar, tDiskData in Mgrid.tDiskData.groupby(level=0)}

In [343]:
# just a quick check
group = Mgrid.tDiskData.groupby(level=0)
tDiskData03 = group.get_group('0.3Msol')
tDiskData03.t.describe()

count    9.988000e+03
mean     8.929732e+06
std      4.701408e+06
min      1.298540e+06
25%      5.220575e+06
50%      7.918750e+06
75%      1.172385e+07
max      3.102870e+07
Name: t, dtype: float64

fit exponential function to disk fractions

In [339]:
def exponential(t, A, tau, C):
    return A*np.exp(-t/tau) + C

In [326]:
def fit_diskFractions(times, fractions):
    paramsBounds = ([0.,1e3,-np.inf], [10.,1e9, np.inf])
    params, covariance = curve_fit(exponential, times, fractions, bounds=paramsBounds)
#     params_std = np.sqrt(np.diag(covariance))
    print(params)
#     print(covariance)
#     print('std of parameter errors: {}'.format(params_std))
    
    return params, covariance

In [332]:
def get_fitRangeMask(fractions, f0=0.85, f1=0.01):
    """create mask to constrain range for the fit."""
    return np.ma.masked_inside(fractions, f0, f1)

To parametrize the fraction of remaining disks, we apply a simple exponential fit of the form f(t) = A*exp(-t/tau) + C. There is a clear trend of increasing disk lifetimes with increasing stellar mass. 

In [450]:
fig, ax = plt.subplots()

tau = []
std_tau = []
for Mstar, fractions in diskFractions.items():
    
    # mask times and fit exponential function
    fitRangeMask = get_fitRangeMask(fractions[1])
    fitParams = fit_diskFractions(fractions[0][fitRangeMask.mask], fractions[1][fitRangeMask.mask])
    diskFractions[Mstar] += fitParams
    
    tau.append(fitParams[0][1])
    std_tau.append(np.sqrt(np.diag(fitParams[1]))[1])
    
    # plot fractions
    ax.scatter(fractions[0], fractions[1], marker='x', label=Mstar[:-4] + r'$\mathrm{M_\odot}$' + r', $\tau = {:1.1f}$ Myr'.format(tau[-1]/1e6))
    
    # plot fits
    ax.plot(fractions[0][fitRangeMask.mask], exponential(fractions[0][fitRangeMask.mask], 
            fitParams[0][0], fitParams[0][1], fitParams[0][2]))
#     ax.set_xscale('log')
plt.legend()

ax.axhline(.5, ls='--')

ax.set_xlabel('Time [yr]')
ax.set_ylabel('Disk Fraction')
plt.show()

[  1.74768871e+00   7.04689235e+06  -7.93697370e-02]
[  1.73571094e+00   9.37474430e+06  -8.82690210e-02]
[  1.71768957e+00   1.12617687e+07  -1.01268908e-01]
[  1.76474029e+00   1.38840446e+07  -1.51234559e-01]


what is the trend in 'tau'?

In [413]:
def linear(x, a, b):
    y = a*np.array(x) + b
    return y

In [448]:
Mstar = [float(key[:-4]) for key in diskFractions.keys()]
tau = [m[2][1] for m in diskFractions.values()]

params, covariance = curve_fit(linear, Mstar, tau, sigma=np.array(std_tau))
pErr = np.sqrt(np.diag(covariance))

# prepare confidence intervals
nSigma = 1.
pHi = params + nSigma*pErr
pLo = params - nSigma*pErr

fit = linear(Mstar, *params)
fitHi = linear(Mstar, *pHi)
fitLo = linear(Mstar, *pLo)

fig, ax = plt.subplots()
(_, caps, _) = ax.errorbar(Mstar, tau, std_tau, lw=2, capsize=5, fmt='o', label=r'$\tau ( \mathrm{M_\star})$')
ax.plot(Mstar, fit, c='gray', label=r'$\tau = ({:1.1f}*\mathrm{{M_\star}} + {:1.1f}$) Myr'.format(*params/1e6))
ax.fill_between(Mstar, fitHi, fitLo, alpha=.2, label=r'$1\sigma$ confidence interval')


# dirty hack to recover errorbar caps
for cap in caps:
    cap.set_markeredgewidth(1)

ax.set_xlabel(r'Host Star Mass $[M_\odot]$')
ax.set_ylabel(r'Time Constant $\tau$')
plt.legend()